In [ ]:
import os
from groq import Groq
import requests
import pandas as pd
from pydantic import BaseModel
import json
import numpy as np

# Testing one Groq API
GROQ_API_KEY = os.environ["GROQ_API_KEY"]

url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

pd.DataFrame(response.json()['data'][:5])

,id,object,created,owned_by,active,context_window,public_apps
0,whisper-large-v3,model,1693721698,OpenAI,True,448,None
1,llama-3.2-90b-vision-preview,model,1727226914,Meta,True,8192,None
2,llama-3.2-90b-text-preview,model,1727285716,Meta,True,8192,None
3,llama3-groq-8b-8192-tool-use-preview,model,1693721698,Groq,True,8192,None
4,llava-v1.5-7b-4096-preview,model,1725402373,Other,True,4096,None


In [35]:
client = Groq()

# Model for tool-use
MODEL = 'llama-3.1-70b-versatile'

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

In [36]:
def run_conversation(user_prompt):
    # Initialize the conversation
    messages = [
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform calculations."
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

    # Define the available functions/tools for our model to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate in python 3.11. You have access to numpy as np.",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]

    # Make the initial API call to Groq
    response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        stream=False,
        tools=tools, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_tokens=4096 # Maximum number of tokens to allow in our response
    )

    # Extract the response and any tool call responses
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
            # Define the available tools that can be called by the LLM
            available_functions = {
                "calculate": calculate,
            }
            # Add the LLM's response to the conversation
            messages.append(response_message)

            # Process each tool call
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                # Call the tool and get the response
                function_response = function_to_call(
                    expression=function_args.get("expression")
                )
                # Add the tool response to the conversation
                messages.append(
                    {
                        "tool_call_id": tool_call.id, 
                        "role": "tool", # Indicates this message is from tool use
                        "name": function_name,
                        "content": function_response,
                    }
                )
            # Make a second API call with the updated conversation
            second_response = client.chat.completions.create(
                model=MODEL,
                messages=messages
            )
            # Return the final response
            return second_response.choices[0].message.content, messages
    

prompt = "What is 27970.928461529482 squared?"

res, messages = run_conversation(prompt)


In [37]:
messages

[{'role': 'system',
  'content': 'You are a calculator assistant. Use the calculate function to perform calculations.'},
 {'role': 'user', 'content': 'What is 27970.928461529482 squared?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fccy', function=Function(arguments='{"expression": "27970.928461529482 ** 2"}', name='calculate'), type='function')]),
 {'tool_call_id': 'call_fccy',
  'role': 'tool',
  'name': 'calculate',
  'content': '{"result": 782372839.0}'}]

In [38]:
print(res)

The result of 27970.928461529482 squared is 782,372,839.
